In [ ]:
import tkinter as tk
from tkinter import ttk, simpledialog
import datetime
import threading
import time
from queue import Queue

class OnlineShoppingMonitor:
    def __init__(self):
        self.products = {
            "Shirt": {"stock": 10, "status": "Available", "sem": threading.Semaphore(1)},
            "Shoes": {"stock": 15, "status": "Available", "sem": threading.Semaphore(1)},
            "Jeans": {"stock": 20, "status": "Available", "sem": threading.Semaphore(1)},
            "Hat": {"stock": 12, "status": "Available", "sem": threading.Semaphore(1)},
        }
        self.max_concurrent_users = 3
        self.sem = threading.Semaphore(self.max_concurrent_users)

    def add_product(self, name, stock):
        if name in self.products or stock <= 0:
            return False
        self.products[name] = {"stock": stock, "status": "Available", "sem": threading.Semaphore(1)}
        return True

class OnlineShoppingSimulation:
    def __init__(self, root):
        self.root = root
        self.root.title("Online Shopping Simulator")
        self.root.geometry("1200x700")
        self.monitor = OnlineShoppingMonitor()
        self.selected_products = {i: [] for i in range(5)}
        self.order_queue = Queue()
        self.running = True
        self.threads = []
        self.setup_main_gui()
        self.root.protocol("WM_DELETE_WINDOW", self.on_exit)

    def setup_main_gui(self):
        self.header_frame = tk.Frame(self.root, bg="#2c3e50")
        self.header_frame.pack(fill="x", pady=10)

        self.header_label = tk.Label(
            self.header_frame,
            text="Welcome to the Online Shopping Simulator!",
            font=("Arial", 24, "bold"),
            fg="white",
            bg="#2c3e50",
        )
        self.header_label.pack(side="left", padx=20)

        self.button_frame = tk.Frame(self.root, bg="#ecf0f1")
        self.button_frame.pack(fill="x", pady=5)

        self.add_product_button = self.create_button(self.button_frame, text="Add Product", command=self.add_product_dialog)
        self.add_product_button.grid(row=0, column=0, padx=2, pady=2, sticky="w")

        # Removed the "Remove Product" button
        # self.remove_product_button = self.create_button(self.button_frame, text="Remove Product", command=self.remove_product_dialog)
        # self.remove_product_button.grid(row=0, column=1, padx=2, pady=2, sticky="w")

        self.place_order_button = self.create_button(self.button_frame, text="Place Order", command=self.place_order)
        self.place_order_button.grid(row=0, column=1, padx=2, pady=2, sticky="w")

        self.setup_gui_components()

    def setup_gui_components(self):
        self.main_frame = tk.Frame(self.root)
        self.main_frame.pack(fill="both", expand=True, padx=20)

        self.main_frame.grid_rowconfigure(0, weight=1)
        self.main_frame.grid_rowconfigure(1, weight=1)
        self.main_frame.grid_rowconfigure(2, weight=1)
        self.main_frame.grid_columnconfigure(0, weight=1)
        self.main_frame.grid_columnconfigure(1, weight=2)

        self.selection_frame = tk.Frame(self.main_frame, bg="#34495e")
        self.selection_frame.grid(row=0, column=0, padx=5, pady=5, sticky="nsew")

        select_label = tk.Label(self.selection_frame, text="Select the Products", font=("Arial", 20, "bold"), fg="white", bg="#34495e")
        select_label.grid(row=0, column=0, columnspan=5, pady=5, padx=5, sticky="w")

        for i in range(5):
            frame = tk.Frame(self.selection_frame, bg="#34495e", padx=5, pady=5)
            frame.grid(row=1, column=i, padx=5, pady=5)

            label = tk.Label(frame, text=f"User {i + 1}", font=("Arial", 10), fg="white", bg="#34495e")
            label.pack(pady=5)

            product_listbox = tk.Listbox(frame, selectmode=tk.MULTIPLE, height=3, font=("Arial", 10), bg="#ecf0f1", fg="black", bd=0)
            for product in self.monitor.products.keys():
                product_listbox.insert(tk.END, product)
            product_listbox.pack(pady=5)

            add_to_cart_button = self.create_button(frame, text="Add to Cart", command=lambda index=i: self.add_to_cart(index))
            add_to_cart_button.pack(pady=5)

            remove_from_cart_button = self.create_button(frame, text="Remove from Cart", command=lambda index=i: self.remove_from_cart(index))
            remove_from_cart_button.pack(pady=5)

            user_cart_listbox = tk.Listbox(frame, height=3, font=("Arial", 8), bg="#ecf0f1", fg="black", bd=0)
            user_cart_listbox.pack(pady=5)

            setattr(self, f"product_listbox_{i + 1}", product_listbox)
            setattr(self, f"user_cart_listbox_{i + 1}", user_cart_listbox)

        self.critical_section_frame = tk.Frame(self.main_frame, bg="#34495e")
        self.critical_section_frame.grid(row=1, column=0, padx=5, pady=5, sticky="nsew")

        critical_section_label = tk.Label(
            self.critical_section_frame, text="Critical Section Activity", font=("Arial", 14, "bold"), fg="white", bg="#34495e"
        )
        critical_section_label.pack(pady=5)

        self.partitioned_frame = tk.Frame(self.critical_section_frame, bg="#2c3e50", bd=2, relief="ridge")
        self.partitioned_frame.pack(fill="both", expand=True, padx=10, pady=10)

        self.active_users_box = tk.Listbox(self.partitioned_frame, font=("Arial", 10), bg="#2c3e50", fg="white", bd=0, height=7)
        self.active_users_box.grid(row=0, column=0, padx=5, pady=5, sticky="nsew")

        self.completed_users_box = tk.Listbox(self.partitioned_frame, font=("Arial", 10), bg="#2c3e50", fg="white", bd=0, height=7)
        self.completed_users_box.grid(row=0, column=1, padx=5, pady=5, sticky="nsew")

        self.partitioned_frame.grid_columnconfigure(0, weight=1)
        self.partitioned_frame.grid_columnconfigure(1, weight=1)

        self.available_products_frame = tk.Frame(self.main_frame, bg="#34495e")
        self.available_products_frame.grid(row=2, column=0, padx=5, pady=5, sticky="nsew")

        available_label = tk.Label(self.available_products_frame, text="Available Products", font=("Arial", 19, "bold"), fg="white", bg="#34495e")
        available_label.grid(row=0, column=0, columnspan=3, pady=5, padx=5, sticky="w")

        self.product_tree = ttk.Treeview(self.available_products_frame, columns=("Product", "Stock", "Status"), show="headings", height=4)
        self.product_tree.heading("Product", text="Product", anchor="w")
        self.product_tree.heading("Stock", text="Stock", anchor="center")
        self.product_tree.heading("Status", text="Status", anchor="center")
        self.product_tree.grid(row=1, column=0, padx=5, pady=5, sticky="nsew")
        self.refresh_product_tree()

        self.system_log_frame = tk.Frame(self.main_frame, bg="#34495e")
        self.system_log_frame.grid(row=0, column=1, rowspan=3, padx=10, pady=5, sticky="nsew")

        self.log_label = tk.Label(self.system_log_frame, text="System Log", font=("Arial", 12, "bold"), fg="white", bg="#34495e")
        self.log_label.pack(pady=5)

        self.log_text = tk.Text(self.system_log_frame, wrap="word", height=12, width=40, state="disabled", bg="#34495e", fg="white", font=("Courier", 8))
        self.log_text.pack(fill="both", expand=True, padx=5, pady=5)

    def create_button(self, parent, text, command):
        return tk.Button(parent, text=text, font=("Arial", 10, "bold"), command=command, relief="flat", bg="#3498db", fg="white")

    def refresh_product_tree(self):
        self.product_tree.delete(*self.product_tree.get_children())
        for product, details in self.monitor.products.items():
            self.product_tree.insert("", "end", values=(product, details["stock"], details["status"]))

    def refresh_product_listboxes(self):
        for i in range(5):
            listbox = getattr(self, f"product_listbox_{i + 1}")
            listbox.delete(0, tk.END)
            for product in self.monitor.products.keys():
                listbox.insert(tk.END, product)

    def add_to_cart(self, user_index):
        listbox = getattr(self, f"product_listbox_{user_index + 1}")
        cart_listbox = getattr(self, f"user_cart_listbox_{user_index + 1}")
        selected_items = listbox.curselection()
        for item in selected_items:
            product = listbox.get(item)
            if product not in self.selected_products[user_index]:
                self.selected_products[user_index].append(product)
                cart_listbox.insert(tk.END, product)

    def remove_from_cart(self, user_index):
        cart_listbox = getattr(self, f"user_cart_listbox_{user_index + 1}")
        selected_items = cart_listbox.curselection()
        for item in reversed(selected_items):
            product = cart_listbox.get(item)
            self.selected_products[user_index].remove(product)
            cart_listbox.delete(item)

    def add_product_dialog(self):
        name = simpledialog.askstring("Add Product", "Enter the product name:")
        stock = simpledialog.askinteger("Add Product", "Enter the stock quantity:")
        if not name or stock is None:
            return
        if stock <= 0:
            self.log_message(f"Error: Invalid stock value for product '{name}'. Stock must be greater than 0.")
            return
        if self.monitor.add_product(name, stock):
            self.log_message(f"Product '{name}' added with stock: {stock}.")
            self.refresh_product_tree()
            self.refresh_product_listboxes()
        else:
            self.log_message(f"Error: Product '{name}' already exists or invalid stock.")

    def place_order(self):
        empty_orders = True
        for i in range(5):
            if self.selected_products[i]:
                empty_orders = False
                products = self.selected_products[i].copy()
                self.selected_products[i] = []  # Clear the user's cart after placing order
                self.log_message(f"User {i + 1} has placed an order for {', '.join(products)}.")
                self.order_queue.put((i, products))  # Put the order in the queue
        if empty_orders:
            self.log_message("Error: No items in the cart for any user. Order not placed.")
        else:
            threading.Thread(target=self.process_orders_from_queue, daemon=True).start()

    def process_orders_from_queue(self):
        while not self.order_queue.empty() and self.running:
            user_index, products = self.order_queue.get()
            self.monitor.sem.acquire()  # Limit to max concurrent users
            thread = threading.Thread(target=self.process_order, args=(user_index, products), daemon=True)
            self.threads.append(thread)
            thread.start()

    def process_order(self, user_index, products):
        for product in products:
            if not self.running:
                return  # Exit if the app is not running
            product_data = self.monitor.products.get(product)
            if product_data:
                product_data["sem"].acquire()
                self.add_to_active_users(f"User {user_index + 1} is processing '{product}'.")
                time.sleep(2)  # Simulate processing time
                if product_data["stock"] > 0:
                    product_data["stock"] -= 1
                    product_data["status"] = "Available" if product_data["stock"] > 0 else "Out of Stock"
                    self.log_message_safe(f"User {user_index + 1} purchased '{product}'. Stock left: {product_data['stock']}.")
                else:
                    self.log_message_safe(f"User {user_index + 1} could not purchase '{product}' (Out of Stock).")
                product_data["sem"].release()
                self.move_to_completed_users(f"User {user_index + 1} finished processing '{product}'.")

        self.refresh_product_tree_safe()
        self.monitor.sem.release()

    def add_to_active_users(self, message):
        self.root.after(0, lambda: self.active_users_box.insert(tk.END, message))
        self.root.after(0, self.active_users_box.yview, tk.END)

    def move_to_completed_users(self, message):
        self.root.after(0, lambda: self.completed_users_box.insert(tk.END, message))
        self.root.after(0, lambda: self.active_users_box.delete(0))  # Remove the first active user entry
        self.root.after(0, self.completed_users_box.yview, tk.END)

    def log_message_safe(self, message):
        self.root.after(0, self.log_message, message)

    def refresh_product_tree_safe(self):
        self.root.after(0, self.refresh_product_tree)

    def log_message(self, message):
        timestamp = datetime.datetime.now().strftime("%H:%M:%S")
        full_message = f"{timestamp} - {message}\n"
        self.log_text.configure(state="normal")
        self.log_text.insert("end", full_message)
        self.log_text.configure(state="disabled")
        self.log_text.see("end")

    def on_exit(self):
        """Handle application exit and clean up threads."""
        self.running = False  # Stop all background processes
        for thread in self.threads:
            thread.join(timeout=1)  # Wait for threads to finish
        self.root.destroy()


if __name__ == "__main__":
    root = tk.Tk()
    app = OnlineShoppingSimulation(root)
    root.mainloop()